In [2]:
import pandas as pd
df1 = pd.read_csv('submission_example.csv')
df2 = pd.read_csv('x_test.csv')
df3 = pd.read_csv('x_train.csv')
df4 = pd.read_csv('y_train.csv')

In [3]:
def norm_y(y):
    new_y = y.rename(columns={'id': 'id', 'Глубина  проникания иглы при 0 °С, [мм-1]': 'depth_0',
                              'Глубина  проникания иглы при 25 °С, [мм-1]': 'depth_25',
                              'Растяжимость  при температуре 0 °С, [см]': 'laxity',
                              'Температура размягчения, [°С]': 'softening', 'Эластичность при 0 °С, [%]': 'elasticity'})
    y=y.drop(['id'], axis=1)
    return new_y

In [4]:
from sklearn.preprocessing import StandardScaler

def norm_x(x):
    x = x.rename(columns={'id': 'id', '% массы <Адгезионная добавка>': 'adhesive_add', '% массы <Базовый битум>': 'bitumen',
       '% массы <Пластификатор>': 'plasticizer', '% массы <Полимер>': 'polymer',
       '% массы <Сшивающая добавка>': 'cross-linking_add', 'Исходная игла при 25С <Базовый битум>': 'bitumen_25',
       'Адгезионная добавка': 'Адгезионная добавка', 'Пластификатор': 'Пластификатор', 'Полимер': 'Полимер',
       'Базовая пенетрация для расчёта пластификатора': 'basic_penetration',
       'Расчёт рецептуры на глубину проникания иглы при 25': 'calculation_25'})
    x=x.drop(['id'], axis=1)
    x_polymer = pd.get_dummies(x['Полимер'])
    x_plasticizer =pd.get_dummies(x['Пластификатор'])
    x_adhesion = pd.get_dummies(x['Адгезионная добавка'])
    x = x.drop(['Полимер', 'Пластификатор', 'Адгезионная добавка'], axis=1)
    x = pd.concat([x, x_polymer, x_plasticizer, x_adhesion], axis=1)
    return x

In [12]:
norm_x(df3)

adhesive_add  bitumen  plasticizer  polymer  cross-linking_add  \
0           0.000   90.400        5.900    3.600              0.100   
1           0.300   90.100        5.900    3.600              0.100   
2           0.300   90.100        5.900    3.600              0.100   
3           0.300   90.100        5.900    3.600              0.100   
4           0.300   89.000        7.000    3.600              0.100   
..            ...      ...          ...      ...                ...   
103         0.300   89.944        5.899    3.749              0.108   
104         0.300   88.925        6.917    3.750              0.108   
105         0.293   91.142        5.206    3.270              0.089   
106         0.311   91.652        4.040    3.885              0.112   
107         0.000   90.850        5.501    3.549              0.100   

     bitumen_25  basic_penetration  calculation_25  Polymer1  Polymer2  ...  \
0            79                138              75         1         0  ...   
1            79                138              75         0         1  ...   
2            79                138              75         0         1  ...   
3            79                138              75         0         1  ...   
4            79                138              75         0         1  ...   
..          ...                ...             ...       ...       ...  ...   
103          79                139              75         0         1  ...   
104          71                105              75         0         1  ...   
105          72                103              75         0         1  ...   
106          79                118              75         0         1  ...   
107          80                135              75         0         1  ...   

     Plasticizer5  Plasticizer6  Plasticizer7  Plasticizer8  Plasticizer9  \
0               0             0             0             0             0   
1               0             0             0             0             0   
2               0             0             0             0             0   
3               0             0             0             0             0   
4               0             0             0             0             0   
..            ...           ...           ...           ...           ...   
103             0             0             0             0             0   
104             0             0             0             0             0   
105             0             0             0             0             0   
106             0             0             0             0             0   
107             0             0             0             0             0   

     Adhesion1  Adhesion2  Adhesion3  Adhesion4  Отсутствует  
0            0          0          0          0            1  
1            1          0          0          0            0  
2            1          0          0          0            0  
3            1          0          0          0            0  
4            1          0          0          0            0  
..         ...        ...        ...        ...          ...  
103          1          0          0          0            0  
104          1          0          0          0            0  
105          1          0          0          0            0  
106          0          0          0          1            0  
107          0          0          0          0            1  

[108 rows x 42 columns]

In [25]:
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif, SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import linear_model
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.svm import SVR

In [26]:
def lr1(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = GradientBoostingRegressor(random_state=0).fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [27]:
lr1, ml1 = lr1(df2, df3, df4, 'depth_0')

In [28]:
def lr2(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = LinearRegression().fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [29]:
lr2, ml2 = lr2(df2, df3, df4, 'depth_25')

In [30]:
def lr3(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = GradientBoostingRegressor(random_state=0).fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [31]:
lr3, ml3 = lr3(df2, df3, df4, 'laxity')

In [32]:
def lr4(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = GradientBoostingRegressor(random_state=0).fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [33]:
lr4, ml4 = lr4(df2, df3, df4, 'softening')

In [34]:
def lr5(df2, df3, df4, params):
    y=norm_y(df4)
    x=norm_x(df3)
    x_test = norm_x(pd.concat([df3, df2]))
    x_test = x_test[-10:]
    
    x_data_LinearRegression = x.loc[y[params].notnull()]#признаки для модели
    y_LinearRegression = y[params].loc[y[params].notnull()] #значения для модели
    reg = SVR(C=10).fit(x_data_LinearRegression, y_LinearRegression)
    y_predict = reg.predict(x_test)
    return y_predict, reg

In [35]:
lr5, ml5 = lr5(df2, df3, df4, 'elasticity')

In [36]:
id = [0,1,2,3,4,5,6,7,8,9]
res1 = pd.DataFrame()
res1['id'] = id
res1['Глубина  проникания иглы при 0 °С, [мм-1]'] = lr1
res1['Глубина  проникания иглы при 25 °С, [мм-1]'] = lr2
res1['Растяжимость  при температуре 0 °С, [см]'] = lr3
res1['Температура размягчения, [°С]'] = lr4
res1['Эластичность при 0 °С, [%]'] = lr5

In [37]:
res1

id  Глубина  проникания иглы при 0 °С, [мм-1]  \
0   0                                  29.655014   
1   1                                  33.494292   
2   2                                  32.889541   
3   3                                  32.407595   
4   4                                  32.458536   
5   5                                  31.991640   
6   6                                  30.238297   
7   7                                  32.765147   
8   8                                  36.418071   
9   9                                  31.713522   

   Глубина  проникания иглы при 25 °С, [мм-1]  \
0                                   70.310280   
1                                  105.313928   
2                                   65.207972   
3                                   83.283071   
4                                   68.336876   
5                                   60.163738   
6                                   61.126375   
7                                   60.314820   
8                                   96.280451   
9                                   66.075693   

   Растяжимость  при температуре 0 °С, [см]  Температура размягчения, [°С]  \
0                                 15.559729                      61.318046   
1                                 46.005585                      59.020578   
2                                 31.024731                      70.379028   
3                                 43.872450                      69.440191   
4                                 30.860426                      74.110536   
5                                 24.214513                      75.740703   
6                                 19.665540                      75.779686   
7                                 24.275503                      63.982477   
8                                 31.774872                      57.910623   
9                                 22.472624                      63.666113   

   Эластичность при 0 °С, [%]  
0                   73.344342  
1                   76.469658  
2                   74.123161  
3                   74.756470  
4                   74.357318  
5                   73.490370  
6                   73.375589  
7                   73.136543  
8                   72.771601  
9                   72.899845

In [38]:
res1.to_csv(f'full_svr5.csv', index=False)

In [39]:
import pickle
with open('ml1.pkl', 'wb') as f:
    pickle.dump(ml1, f, pickle.HIGHEST_PROTOCOL)
with open('ml2.pkl', 'wb') as f:
    pickle.dump(ml2, f)
with open('ml3.pkl', 'wb') as f:
    pickle.dump(ml3, f)
with open('ml4.pkl', 'wb') as f:
    pickle.dump(ml4, f)
with open('ml5.pkl', 'wb') as f:
    pickle.dump(ml5, f)